<table>
<tr>
    <td>
        <img src="https://www.wordstream.com/wp-content/uploads/2021/07/how-to-get-amazon-reviews.png" width="200"/>
    </td>
    <td style="text-align: left; vertical-align: top;">
        <h1><strong>Amazon Reviews</strong><br></h1>
        <h4>Engineering Large Scale Data Analytics Systems<br>
        ENSF 612 - Fall 2023</h4>
    </td>
</tr>
</table>


*** Note: run all the code the first time. For subsecuent runs, you can set the individual process flags below to False. This will avoid resetting spark, reloading the datasets or repeat computing intensive tasks that were previously computed and stored.


In [6]:
set_spark = True
load_datasets = True
inspect_data = True
pre_process = True

**Setting Up Spark, Spark NLP and Required Modules**

In [7]:
# the capture magic command captures the output of the block to avoid clutter
%%capture

if set_spark:
  !apt-get install openjdk-8-jdk-headless -qq > /dev/null
  !wget https://dlcdn.apache.org/spark/spark-3.3.3/spark-3.3.3-bin-hadoop3.tgz
  !tar -xvf spark-3.3.3-bin-hadoop3.tgz
  !pip install findspark
  !pip install -q spark-nlp
  !pip install contractions

  import os
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
  os.environ["SPARK_HOME"] = "/content/spark-3.3.3-bin-hadoop3"

  import findspark
  findspark.init()
  findspark.find()
  from pyspark.sql import SparkSession
  import sparknlp

  # Setting up 4 threads, potentially allowing a 4-core processor execute 4 tasks in parallel
  # And adding the Spark NLP package to the Spark session
  spark = SparkSession.builder\
      .appName("Colab")\
      .master("local[4]")\
      .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:5.1.4")\
      .getOrCreate()

  sc = spark.sparkContext

**Cloning Github Repository and Loading Datasets**

In [8]:
if load_datasets:
  !git clone https://github.com/MENG2023-TP/ENSF612-Project.git

fatal: destination path 'ENSF612-Project' already exists and is not an empty directory.


In [9]:
!ls ENSF612-Project/datasets

All_Beauty_5.json  Cell_Phones_and_Accessories_5_subsample.json  Software_5.json
Appliances_5.json  Musical_Instruments_5_subsample.json		 Video_Games_5_subsample.json


In [10]:
dataset_directory = 'ENSF612-Project/datasets'

# Gets the list of files in the dataset directory that end in ".json"
json_files = [file for file in os.listdir(dataset_directory) if file.endswith('.json')]

# Creates a list of full file paths
file_paths = [os.path.join(dataset_directory, file) for file in json_files]

In [11]:
import json

# Function to parse NDJSON (new line-delimited JSON) files and extract specific fields
def parse_ndjson(line):
    try:
        # Parse the JSON line and return only reviewText asin and reviewerID
        json_line = json.loads(line)
        return (
            json_line.get('overall', ''),
            json_line.get('reviewText', '')
        )
    except json.JSONDecodeError:
        # In case of error, skip this record and return None
        return None

In [12]:
if load_datasets:
  # Initialize an empty RDD
  data_rdd = spark.sparkContext.emptyRDD()

  # Read each file into an RDD, parse its ndjson objects if not None, and union with the existing RDD
  for file_path in file_paths:
      file_rdd = sc.textFile(file_path, 4)
      parsed_rdd = file_rdd.map(parse_ndjson).filter(lambda x: x is not None)
      data_rdd = data_rdd.union(parsed_rdd)

  # convert the data_rdd to a distributed Spark DataFrame
  df = spark.createDataFrame(data_rdd, schema=['score', 'review']).cache()

**Data Inspection**

In [13]:
type(df) # Shows type of df

pyspark.sql.dataframe.DataFrame

In [14]:
df # Shows attributes of df

DataFrame[score: double, review: string]

In [15]:
if inspect_data:
  count = df.count() # Count amount of records in the DataFrame
count

118242

In [16]:
df.take(1) # Preview a single record

[Row(score=5.0, review="Grand Theft Auto III is quite possibly the best game ever created.  With a fully-diverse world complete with po-po, various gangs and affiliations; each with their own specific type of car that they drive, angry pedestrians, mobs, thiefs, innocent people, various cars ranging from nice sports cars to crappy Volkswagen esque vehicles, to mack trucks, to pick up trucks, delivery vans, buses, tanks, taxis, speed and police boats...whatever... you name it.\nYou can virtually go anywhere in the humongous city and either complete missions or wreak havoc on pedestrians.  Both objectives are equally fun.  You can get a kick out of standing a top a roof or parking garage and sniping the innocent pedestrians below, killing and fighting various gang members and watching their buddies come help them out, start a fight with a random pedestrian who says something you don't particularly like when you walk by, defy the cops etc. etc.  Steal a taxi and earn money dropping off di

In [17]:
if inspect_data:
  from pyspark.sql.functions import col, sum
  null_count = df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns]).cache()  # Count null values

In [18]:
null_count.show()

+-----+------+
|score|review|
+-----+------+
|    0|     0|
+-----+------+



**Text Pre-Processing**

Expanding contractions

Although this step in not estrictly neccesary. Expanding contractions can make the text clearer and more consistent for the model, which can improve its ability to interpret and analyze the words.

In [19]:
if pre_process:
  from pyspark.sql.functions import udf
  from pyspark.sql.types import StringType
  import contractions

  # Define the UDF for expanding contractions
  def expand_contractions_text(text):
      return contractions.fix(text)

  expand_contractions_udf = udf(expand_contractions_text, StringType())

  # Apply the UDF to the DataFrame to create a new column with expanded contractions
  expanded_df = df.withColumn("expanded_review", expand_contractions_udf("review"))

Defining DocumentAssembler and Spark NLP components

The DocumentAssembler is the initial step in a Spark NLP pipeline. It converts raw text into a structured Annotation format that subsequent Spark NLP annotators can utilize for processing.

In [20]:
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import Tokenizer, Normalizer, LemmatizerModel, StopWordsCleaner

document_assembler = DocumentAssembler() \
    .setInputCol("expanded_review") \
    .setOutputCol("document")

1. Tokenization

In [21]:
tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

2. Text Cleaning

In [22]:
normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normalized") \
    .setLowercase(True) \
    .setCleanupPatterns(["[^A-Za-z'\\s]"])  # remove punctuations and numbers

3. Stopword Removal

In [23]:
stop_words_cleaner = StopWordsCleaner() \
    .setInputCols(["normalized"]) \
    .setOutputCol("cleanTokens")

4. Stemming/Lemmatization.

Stemming and lemmatization are both text normalization techniques that reduce words to their base or root form. Applying both can at times be redundant. For this application we decide to use Lemmatization.


In [24]:
# Use the pretrained LemmatizerModel from Spark NLP
lemmatizer = LemmatizerModel.pretrained() \
    .setInputCols(["cleanTokens"]) \
    .setOutputCol("lemmatized")

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[OK!]


In [25]:
if pre_process:
  # Define the Spark NLP pipeline
  from pyspark.ml import Pipeline

  pipeline = Pipeline(stages=[
      document_assembler,
      tokenizer,
      normalizer,
      stop_words_cleaner,
      lemmatizer
  ])

  processed = pipeline.fit(expanded_df).transform(expanded_df).cache()

In [26]:
# Show the processed data
processed.show()

+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|score|              review|     expanded_review|            document|               token|          normalized|         cleanTokens|          lemmatized|
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|  5.0|Grand Theft Auto ...|Grand Theft Auto ...|[{document, 0, 40...|[{token, 0, 4, Gr...|[{token, 0, 4, gr...|[{token, 0, 4, gr...|[{token, 0, 4, gr...|
|  4.0|I surprised my so...|I surprised my so...|[{document, 0, 10...|[{token, 0, 0, I,...|[{token, 0, 0, i,...|[{token, 2, 10, s...|[{token, 2, 10, s...|
|  5.0|Great value. Buy ...|Great value. Buy ...|[{document, 0, 19...|[{token, 0, 4, Gr...|[{token, 0, 4, gr...|[{token, 0, 4, gr...|[{token, 0, 4, gr...|
|  5.0|Of the 8 or so Wi...|Of the 8 or so Wi...|[{document, 0, 51...|

In [27]:
# Get the first row of the DataFrame
first_row = processed.first()

# Print first row with its content
print("Score:", first_row['score'])
print("Review:", first_row['review'])
print("Document:", [doc.result for doc in first_row['document']])
print("Token:", [tok.result for tok in first_row['token']])
print("Normalized:", [norm.result for norm in first_row['normalized']])
print("Clean Tokens:", [clean.result for clean in first_row['cleanTokens']])
print("Lemmatized:", [lemma.result for lemma in first_row['lemmatized']])


Score: 5.0
Review: Grand Theft Auto III is quite possibly the best game ever created.  With a fully-diverse world complete with po-po, various gangs and affiliations; each with their own specific type of car that they drive, angry pedestrians, mobs, thiefs, innocent people, various cars ranging from nice sports cars to crappy Volkswagen esque vehicles, to mack trucks, to pick up trucks, delivery vans, buses, tanks, taxis, speed and police boats...whatever... you name it.
You can virtually go anywhere in the humongous city and either complete missions or wreak havoc on pedestrians.  Both objectives are equally fun.  You can get a kick out of standing a top a roof or parking garage and sniping the innocent pedestrians below, killing and fighting various gang members and watching their buddies come help them out, start a fight with a random pedestrian who says something you don't particularly like when you walk by, defy the cops etc. etc.  Steal a taxi and earn money dropping off differen

**Feature extraction**

Next steps:

Feature extraction (Bag of Words, TF-IDF, word embeddings Word2Vec)

Vectorization (Count Vectorizer, TfidfVectorizer)

Model selection (LogisticRegression, Nayve Bayes, SVM or unsupervised learning)





Additional steps (optional to improve accuracy):

Speech tagging (before stop word removal)

N-grams to use along with TD-IDF